#Pinns

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model



ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

# Define the SIR model parameters
beta = 0.3  # Infection rate
gamma = 0.1  # Recovery rate

# Generate synthetic data for training
def generate_data(N=1000):
    t = np.linspace(0, 10, N)
    S = np.exp(-beta * t)
    I = (beta / gamma) * (1 - np.exp(-gamma * t))
    R = 1 - S - I
    return t, S, I, R

t, S, I, R = generate_data()

# Define the PINN model
input_layer = Input(shape=(1,))
hidden = Dense(20, activation="tanh")(input_layer)
hidden = Dense(20, activation="tanh")(hidden)
output_layer = Dense(3, activation="linear")(hidden)
model = Model(inputs=input_layer, outputs=output_layer)

# Define the loss function
def pinn_loss(t, S_true, I_true, R_true):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(t)
        outputs = model(t)
        S_pred, I_pred, R_pred = tf.split(outputs, num_or_size_splits=3, axis=1)
        dS_dt = tape.gradient(S_pred, t)
        dI_dt = tape.gradient(I_pred, t)
        dR_dt = tape.gradient(R_pred, t)

    # SIR equations as constraints
    eq1 = dS_dt + beta * S_pred * I_pred
    eq2 = dI_dt - (beta * S_pred * I_pred - gamma * I_pred)
    eq3 = dR_dt - gamma * I_pred

    # Combine data loss and physics-based loss
    data_loss = tf.reduce_mean((S_true - S_pred)**2 + (I_true - I_pred)**2 + (R_true - R_pred)**2)
    physics_loss = tf.reduce_mean(eq1**2 + eq2**2 + eq3**2)
    return data_loss + physics_loss

# Compile and train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

@tf.function
def train_step(t, S_true, I_true, R_true):
    with tf.GradientTape() as tape:
        loss = pinn_loss(t, S_true, I_true, R_true)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Training loop
epochs = 1000
for epoch in range(epochs):
    loss = train_step(tf.convert_to_tensor(t[:, None], dtype=tf.float32),
                      tf.convert_to_tensor(S[:, None], dtype=tf.float32),
                      tf.convert_to_tensor(I[:, None], dtype=tf.float32),
                      tf.convert_to_tensor(R[:, None], dtype=tf.float32))
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

print("Training complete.")